In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(rlang)

# Read ready biomarkers data

In [ ]:
r <- fread(paste0(SHARE_DIR, "metaprogram_lm_results.csv"))
d <- fread(paste0(SHARE_DIR, "metaprogram_base_data.csv"))

# Top Results by Type

- All Results

In [ ]:
share <- 
r %>% 
 fi(cohort == "Pan-Cancer", grepl("biopsy", covariate), pval < 1e-2) %>% 
 gb(y) %>% mu(rk = row_number(pval)) %>% ug() %>% 
 ar(pval)

In [ ]:
#share %>% fi(grepl("ZNF750", x)) %>% ar(pval)

- Prioritized

In [ ]:
plts <- share %>% fi(rk < 5) %>% se(y, x)

- Plt function

In [ ]:
plot_custom_boxplot <- function(data, x_var, metaprogram_var) {
  tmp <-   
  data %>% 
   group_by(cohort) %>% 
   mutate(tot = sum(.data[[x_var]])) %>%  # Use .data[[ ]] for column names
   filter(tot > 5) 
 
  nrows <- length(unique(tmp$cohort))
  levels <- length(unique(data[[x_var]]))

  options(repr.plot.width = 14, repr.plot.height = 2.5*nrows)  

  if(levels < 8){  
    base_plt <- 
    tmp %>% ggplot(aes(y = .data[[metaprogram_var]], x = as.factor(.data[[x_var]]), fill = as.factor(.data[[x_var]]))) + 
    geom_boxplot()
  } else {
    base_plt <- 
    tmp %>% ggplot(aes(y = .data[[metaprogram_var]], x = .data[[x_var]])) + 
    geom_point()
  }
  
  plt <- 
  base_plt + 
    facet_grid(cohort ~ biopsy, labeller = label_wrap_gen(width = 16), scales = "free") + 
    go_theme + 
    ggtitle( paste0(metaprogram_var, " vs ", x_var))

  list("plt" = plt, "ht" = 2.5*nrows)  
}

- Go go 

In [ ]:
setwd("/home/josephusset@vhio.org/biomarkers/5_signals/2_metaprograms/1_figures/tmp/")

In [ ]:
for(i in seq(nrow(plts))){
 print(i); flush.console()
 j <- plts[i,]
 a <- plot_custom_boxplot(d, j$x, j$y)

 tryCatch({
  ggsave(paste0(j$y, "_", j$x, ".png"), a$plt, width = 14, height = a$ht)
  }, error = function(e) {
    # Handle the error
    paste("Problem!", j$x, j$y)
  }) }